In [1]:
# import necessary packages
import time
import datetime

import math
import numpy as np
import pandas as pd
import pickle

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import matplotlib
from matplotlib.transforms import Bbox
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages


import light_curve as lc
from itertools import chain

import os
import sys
import shutil
import glob
import json

import warnings
warnings.filterwarnings("ignore")

plt.style.use('fig_publication.mplstyle')

In [2]:


# Specify the path to your JSON file
json_file_path = 'ZTF_sims_from_Alex/ZTFAnomalies_FORPALEO.json'

# Open the JSON file for reading
with open(json_file_path, 'r', encoding='utf-8') as json_file:
    # Initialize an empty list to store parsed data
    data_list = []

    # Iterate over the lines in the file to read and parse each JSON object
    for line in json_file:
        try:
            # Load and parse the JSON object from the current line
            data = json.loads(line)
            
            # Append the parsed data to the list
            data_list.append(data)
        except json.JSONDecodeError as e:
            print(f"Error parsing JSON: {str(e)}")
            continue

# Now, 'data_list' contains all the parsed data from the JSON file


In [3]:
# Assuming 'data_list' contains your parsed JSON data

# Initialize an empty set to store unique keys
all_keys = set()

# Iterate through the parsed data
for item in data_list:
    # Check if the item is a dictionary (JSON object)
    if isinstance(item, dict):
        # Update the set of keys with the keys from this dictionary
        all_keys.update(item.keys())

# Now, 'all_keys' contains all the unique keys present in your JSON data
print(all_keys)


{'MJD', 'HOSTGAL_MAGERR_i', 'HOSTGAL_MAG_z', 'MAGERR', 'HOSTGAL_PHOTOZ', 'SIM_HOSTLIB_GALID', 'HOSTGAL_MAG_i', 'Filter', 'HOSTGAL_MAGERR_r', 'MJD_TRIGGER', 'Type', 'MWEBV', 'HOSTGAL_MAG_r', 'REDSHIFT_HELIO', 'Flux_Err', 'HOSTGAL_MAGERR_g', 'MAG', 'HOSTGAL_PHOTOZ_ERR', 'HOSTGAL_MAGERR_z', 'HOSTGAL_MAG_g', 'HOSTGAL_DDLR', 'Flux'}


Counter({'PISN-STELLA_HYDROGENIC': 978,
         'TDE': 985,
         'KN_B19': 2,
         'SNIIn-MOSFIT': 242,
         'SNIa-91bg': 1023,
         'SNIax': 239,
         'SNIIn+HostXT': 1774,
         'SNII': 2330,
         'CART': 90,
         'PISN-STELLA_HECORE': 595,
         'SLSNI': 4166,
         'PISN-MOSFIT': 4152,
         'SNIb': 989,
         'SNIa': 3530,
         'SNIcBL+HostXT_V19': 1145,
         'SNIc': 2532,
         'ILOT': 1})

In [4]:
uniq_cls_l = np.unique(list(data['Type'].values()))
uniq_cls_l

array(['CART', 'ILOT', 'KN_B19', 'PISN-MOSFIT', 'PISN-STELLA_HECORE',
       'PISN-STELLA_HYDROGENIC', 'SLSNI', 'SNII', 'SNIIn+HostXT',
       'SNIIn-MOSFIT', 'SNIa', 'SNIa-91bg', 'SNIax', 'SNIb', 'SNIc',
       'SNIcBL+HostXT_V19', 'TDE'], dtype='<U22')

In [6]:
uniq_cls_l = ['CART', 'ILOT', 'KN_B19', 'PISN-MOSFIT', 'PISN-STELLA_HECORE',
       'PISN-STELLA_HYDROGENIC', 'SLSNI', 'SNII', 'SNIIn+HostXT',
       'SNIIn-MOSFIT', 'SNIa', 'SNIa-91bg', 'SNIax', 'SNIb', 'SNIc',
       'SNIcBL+HostXT_V19', 'TDE']
uniq_key_l = ['HOSTGAL_MAGERR_g', 'Filter', 'MAGERR', 'HOSTGAL_MAG_i', 'Flux_Err', 
'HOSTGAL_MAG_g', 'MJD', 'MAG', 'HOSTGAL_PHOTOZ', 'MWEBV', 'REDSHIFT_HELIO', 
'HOSTGAL_PHOTOZ_ERR', 'HOSTGAL_MAGERR_z', 'Flux', 'HOSTGAL_DDLR', 'MJD_TRIGGER', 
'Type', 'SIM_HOSTLIB_GALID', 'HOSTGAL_MAG_r', 'HOSTGAL_MAG_z', 'HOSTGAL_MAGERR_i', 'HOSTGAL_MAGERR_r']

for k, v in data.items():
    if k == 'Type':
        for k2, v2 in v.items():
            for cls in uniq_cls_l[1:2]:
                if v2 == cls:
                    for key in uniq_key_l:
                        print(key, data[key].get(k2))

HOSTGAL_MAGERR_g 0.0386699997
Filter ['R', 'R', 'R', 'R', 'R', 'R', 'R']
MAGERR [0.2223860174, 0.2174099684, 0.2562327981, 0.2394263595, 0.1758701801, 0.2411061674, 0.2392552644]
HOSTGAL_MAG_i 18.8571529388
Flux_Err [67.3087234497, 59.2400436401, 64.4442520142, 56.8590278625, 56.1659088135, 54.0352592468, 54.3738861084]
HOSTGAL_MAG_g 20.0341472626
MJD [59113.279, 59116.363, 59118.286, 59134.299, 59136.198, 59140.249, 59143.215]
MAG [21.2083148956, 21.322385788, 21.4093513489, 21.4716567993, 21.1500244141, 21.5345535278, 21.5194034576]
HOSTGAL_PHOTOZ 0.0960194245
MWEBV 0.0252612364
REDSHIFT_HELIO 0.0524141677
HOSTGAL_PHOTOZ_ERR 0.3023099899
HOSTGAL_MAGERR_z 0.0304799993
Flux [328.6046447754, 295.8324279785, 273.060760498, 257.8323059082, 346.7292175293, 243.3205413818, 246.7395172119]
HOSTGAL_DDLR 1.7982153893
MJD_TRIGGER 59113.27734375
Type ILOT
SIM_HOSTLIB_GALID 8000041670
HOSTGAL_MAG_r 19.1736793518
HOSTGAL_MAG_z 18.7136802673
HOSTGAL_MAGERR_i 0.0225699991
HOSTGAL_MAGERR_r 0.02177000